In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [1]:
# Load Packages
import tensorflow as tf
from tensorflow.keras import backend
#from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform
from keras.models import load_model, save_model
import keras
import numpy as np
import pandas as pd
import random
import sys
import io
import re
import os

Using TensorFlow backend.


In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/NLP Group Project/')

In [2]:
# Read Songs
songs = pd.read_csv('drake-songs.csv')

In [3]:
text = ''

for index, row in songs['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [4]:
tokens = text.replace("'", '').split(' ')

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

del char_indices['']
del indices_char[0]

total chars: 5865


In [5]:
maxlen = 15
sentences = []
next_chars = []
for i in range(0, len(tokens)-maxlen-5, 1):
    sentences.append(tokens[i:i+maxlen])
    next_chars.append(tokens[i+maxlen])

In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    y[i, char_indices[next_chars[i]]] = 1
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1

KeyError: ''

In [7]:
model = tf.keras.models.load_model('drake_word_model.h5')

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds[0], 5865)
    return np.argmax(probas)

In [9]:
def generate_text(model, seed, length, diversity):

    maxlen = 15
    generated = ''
    sentence = tokens[seed: seed + maxlen]
    
    for i in sentence:
        generated += str(i)
        generated += ' '

    sentence_list = ''
    for i in sentence:
        sentence_list += str(i) + ' '
    print('----- Generating with seed: "' + sentence_list + '"')
    
    sys.stdout.write(generated)

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence.append(next_char)
        sentence.pop(0)

        sys.stdout.write(next_char)
        sys.stdout.write(' ')

        sys.stdout.flush()

    print()
    print('----- Actual Text -----')
    output = ''
    for token in tokens[seed:seed+maxlen+length]:
        output += str(token) + ' '
    print(output)

In [10]:
generate_text(model, 123, 40, 0.5)

----- Generating with seed: "needs change and im the mofucking cashier nickels for my thoughts dimes in my bed "
needs change and im the mofucking cashier nickels for my thoughts dimes in my bed a i i your back you know the what this and im got a for night know im down do you and and know you you wanna be and like to be you and least and im be should girls 
----- Actual Text -----
needs change and im the mofucking cashier nickels for my thoughts dimes in my bed quarters of the kush shape the lines in my head take my verses too serious ya hate me cause im the one to paint a vivid picture no hd yeah i want it all thats why i strive for it 


Predicted!